In [1]:
import pandas as pd

from processing.tfidf import TfidfResearch
from processing.bm25 import BM25Research
from processing.faiss_index import FaissResearch
from processing.qdrant_index import QdrantResearch

from sqlalchemy import text
from processing.utils import engine
from tqdm.autonotebook import tqdm

from processing.eval import run_and_log
import numpy as np
from processing.utils import clean_text, STOP_RU

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_queries_gt(df: pd.DataFrame) -> tuple[dict[str, list[str]], list[str]]:    
    q_gt: dict[str, list[str]] = {}
    for _, row in df.iterrows():
        q = row["query"]
        art_id = str(row["article_id"])
        q_gt.setdefault(q, []).append(art_id)

    all_ids = df["article_id"].astype(str).unique().tolist()
    return q_gt, all_ids

def queries_by_diff(df: pd.DataFrame) -> dict[str, dict[str, list[str]]]:
    groups: dict[str, dict[str, list[str]]] = {}
    for diff in ("easy", "medium", "hard"):
        sub = df[df["difficulty"] == diff]
        qgt = {q: [str(doc)] for q, doc in zip(sub["query"], sub["article_id"])}
        groups[diff] = qgt

    groups["all"] = {q: [str(doc)] for q, doc in zip(df["query"], df["article_id"])}
    return groups

In [8]:
csv_path_random = "synthetic_data/synthetic_queries_random_10000.csv"
df_synth_random = pd.read_csv(csv_path_random)

In [9]:
queries_gt, article_ids = load_queries_gt(df_synth_random)

q_groups = queries_by_diff(df_synth_random)

print(f"Total requests: {len(queries_gt)}, Documents in corpus: {len(article_ids)}")
print(f"Groups keys: {list(q_groups.keys())}")  # easy, medium, hard, all

Total requests: 29863, Documents in corpus: 10000
Groups keys: ['easy', 'medium', 'hard', 'all']


In [5]:
sql = text("""
    SELECT id::text,
           COALESCE(title, '')   AS title,
           COALESCE(anons, '')   AS anons,
           COALESCE(body, '')    AS body
      FROM public.tmp_news
     WHERE id = ANY(:ids);
""")

df_docs = pd.read_sql(sql, engine, params={"ids": article_ids})


df_docs["full_text"] = (
    df_docs["title"] + ". " +
    df_docs["anons"] + ". " +
    df_docs["body"]
)
texts = df_docs["full_text"].tolist()
ids   = df_docs["id"].tolist()

print("News example:", texts[0][:150], "…")

News example: Американские самолеты мешали российским бомбить игиловцев. Российские самолеты неоднократно сближались в американскими в районе долины реки Евфрат в С …


Base Experiments

In [29]:
tfidf_backend = TfidfResearch(max_features=30_000)
tfidf_backend.index(tqdm(texts), ids)

df_metrics = run_and_log(
    backend      = tfidf_backend,
    q_groups     = q_groups,
    backend_name = "tfidf",
    test_name    = "tfidf_30k_vocab",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

100%|██████████| 167/167 [00:00<00:00, 12217.41it/s]


         test_name backend   level  Precision@10  Recall@10       MRR
0  tfidf_30k_vocab   tfidf    easy      0.097006   0.970060  0.966068
1  tfidf_30k_vocab   tfidf  medium      0.097006   0.970060  0.929142
2  tfidf_30k_vocab   tfidf    hard      0.094578   0.945783  0.892771
3  tfidf_30k_vocab   tfidf     all      0.096200   0.962000  0.929400
4  tfidf_30k_vocab   tfidf     all      0.096200   0.962000  0.929400


In [30]:
bm25_backend = BM25Research()
bm25_backend.index(texts, ids)

df_metrics = run_and_log(
    backend      = bm25_backend,
    q_groups     = q_groups,
    backend_name = "bm25",
    test_name    = "bm25",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

  test_name backend   level  Precision@10  Recall@10       MRR
0      bm25    bm25    easy      0.097605   0.976048  0.968064
1      bm25    bm25  medium      0.097605   0.976048  0.940868
2      bm25    bm25    hard      0.095181   0.951807  0.903614
3      bm25    bm25     all      0.096800   0.968000  0.937583
4      bm25    bm25     all      0.096800   0.968000  0.937583


In [31]:
faiss_berta = FaissResearch(model_name="sergeyzh/BERTA", embed_dim=768)
faiss_berta.index(tqdm(texts), ids)

df_metrics = run_and_log(
    backend      = faiss_berta,
    q_groups     = q_groups,
    backend_name = "faiss_berta",
    test_name    = "Faiss + BERTA, dim=768",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

100%|██████████| 167/167 [00:00<00:00, 6711.85it/s]


                test_name      backend   level  Precision@10  Recall@10  \
0  Faiss + BERTA, dim=768  faiss_berta    easy      0.098802   0.988024   
1  Faiss + BERTA, dim=768  faiss_berta  medium      0.098204   0.982036   
2  Faiss + BERTA, dim=768  faiss_berta    hard      0.096988   0.969880   
3  Faiss + BERTA, dim=768  faiss_berta     all      0.098000   0.980000   
4  Faiss + BERTA, dim=768  faiss_berta     all      0.098000   0.980000   

        MRR  
0  0.958084  
1  0.958583  
2  0.954819  
3  0.957167  
4  0.957167  


In [33]:
faiss_berta = FaissResearch(model_name="sergeyzh/BERTA", embed_dim=1024)
faiss_berta.index(tqdm(texts), ids)

df_metrics = run_and_log(
    backend      = faiss_berta,
    q_groups     = q_groups,
    backend_name = "faiss_berta",
    test_name    = "Faiss + BERTA, dim=1024",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

100%|██████████| 167/167 [00:00<00:00, 11155.24it/s]


                 test_name      backend   level  Precision@10  Recall@10  \
0  Faiss + BERTA, dim=1024  faiss_berta    easy      0.098802   0.988024   
1  Faiss + BERTA, dim=1024  faiss_berta  medium      0.098204   0.982036   
2  Faiss + BERTA, dim=1024  faiss_berta    hard      0.096988   0.969880   
3  Faiss + BERTA, dim=1024  faiss_berta     all      0.098000   0.980000   
4  Faiss + BERTA, dim=1024  faiss_berta     all      0.098000   0.980000   

        MRR  
0  0.958084  
1  0.958583  
2  0.954819  
3  0.957167  
4  0.957167  


In [35]:
qdrant_backend = QdrantResearch(
    collection_name="news_research_synth",
    model_name="sergeyzh/BERTA",
    embed_dim=768,
)
qdrant_backend.index(texts, ids)

df_metrics = run_and_log(
    backend      = qdrant_backend,
    q_groups     = q_groups,
    backend_name = "qdrant_backend",
    test_name    = "QDRANT + BERTA, dim=768",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

                 test_name         backend   level  Precision@10  Recall@10  \
0  QDRANT + BERTA, dim=768  qdrant_backend    easy      0.098802   0.988024   
1  QDRANT + BERTA, dim=768  qdrant_backend  medium      0.098204   0.982036   
2  QDRANT + BERTA, dim=768  qdrant_backend    hard      0.096988   0.969880   
3  QDRANT + BERTA, dim=768  qdrant_backend     all      0.098000   0.980000   
4  QDRANT + BERTA, dim=768  qdrant_backend     all      0.098000   0.980000   

        MRR  
0  0.958084  
1  0.958583  
2  0.953815  
3  0.956833  
4  0.956833  


*Additional experiments*

Lemmatise

In [7]:
import inspect
if not hasattr(inspect, "getargspec"):
    inspect.getargspec = inspect.getfullargspec

from pymorphy3 import MorphAnalyzer 
import nltk, re
nltk.download("punkt", quiet=True)

morph = MorphAnalyzer()

def lemmatize_text(txt: str) -> str:
    tokens = [tok for tok in nltk.word_tokenize(txt.lower()) if tok.isalpha()]
    lemmas = [
        morph.parse(tok)[0].normal_form
        for tok in tokens
        if len(tok) > 2
    ]
    return " ".join(lemmas)


texts_lemma = [lemmatize_text(t) for t in tqdm(texts, desc="pymorphy lemmatize")]
queries_gt_lemma: dict[str, list[str]] = {}

for q, ids_list in tqdm(queries_gt.items(), desc="lemmatize queries"):
    lem_q = lemmatize_text(q)  

    queries_gt_lemma.setdefault(lem_q, []).extend(ids_list)


for k in queries_gt_lemma:
    queries_gt_lemma[k] = list(set(queries_gt_lemma[k]))

lemmatize queries: 100%|██████████| 500/500 [00:00<00:00, 1372.27it/s]


In [9]:
tfidf_lemma = TfidfResearch(max_features=30_000)
tfidf_lemma.index(texts_lemma, ids)

df_metrics = run_and_log(
    backend      = tfidf_lemma,
    q_groups     = q_groups,
    backend_name = "tfidf_lemma",
    test_name    = "tfidf_30k_vocab + lemmatize",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

                     test_name      backend   level  Precision@10  Recall@10  \
0  tfidf_30k_vocab + lemmatize  tfidf_lemma    easy      0.083832   0.838323   
1  tfidf_30k_vocab + lemmatize  tfidf_lemma  medium      0.084431   0.844311   
2  tfidf_30k_vocab + lemmatize  tfidf_lemma    hard      0.085542   0.855422   
3  tfidf_30k_vocab + lemmatize  tfidf_lemma     all      0.084600   0.846000   
4  tfidf_30k_vocab + lemmatize  tfidf_lemma     all      0.084600   0.846000   

        MRR  
0  0.705589  
1  0.703358  
2  0.776520  
3  0.728393  
4  0.728393  


In [10]:
faiss_berta_lemma = FaissResearch(model_name="sergeyzh/BERTA", embed_dim=768)
faiss_berta_lemma.index(texts_lemma, ids)

df_metrics = run_and_log(
    backend      = faiss_berta_lemma,
    q_groups     = q_groups,
    backend_name = "faiss_berta_lemma",
    test_name    = "Faiss + BERTA, dim=768 + lemmatize",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

                            test_name            backend   level  \
0  Faiss + BERTA, dim=768 + lemmatize  faiss_berta_lemma    easy   
1  Faiss + BERTA, dim=768 + lemmatize  faiss_berta_lemma  medium   
2  Faiss + BERTA, dim=768 + lemmatize  faiss_berta_lemma    hard   
3  Faiss + BERTA, dim=768 + lemmatize  faiss_berta_lemma     all   
4  Faiss + BERTA, dim=768 + lemmatize  faiss_berta_lemma     all   

   Precision@10  Recall@10       MRR  
0      0.098802   0.988024  0.936078  
1      0.097605   0.976048  0.948959  
2      0.096988   0.969880  0.950803  
3      0.097800   0.978000  0.945269  
4      0.097800   0.978000  0.945269  


Spellchecker

In [11]:
from spellchecker import SpellChecker
spell = SpellChecker(language="ru")

def spell_correct(txt: str) -> str:
    toks  = txt.split()
    fixed = [spell.correction(t) or t for t in toks]
    return " ".join(fixed)


queries_gt_spell = {spell_correct(q): gts for q, gts in tqdm(queries_gt.items())}

bm25_spell = BM25Research()
bm25_spell.index(texts, ids)

df_metrics = run_and_log(
    backend      = bm25_spell,
    q_groups     = q_groups,
    backend_name = "bm25_spell",
    test_name    = "BM25 + spellchecker",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [06:13<00:00,  1.34it/s]


             test_name     backend   level  Precision@10  Recall@10       MRR
0  BM25 + spellchecker  bm25_spell    easy      0.097006   0.970060  0.967066
1  BM25 + spellchecker  bm25_spell  medium      0.097006   0.970060  0.939870
2  BM25 + spellchecker  bm25_spell    hard      0.094578   0.945783  0.902610
3  BM25 + spellchecker  bm25_spell     all      0.096200   0.962000  0.936583
4  BM25 + spellchecker  bm25_spell     all      0.096200   0.962000  0.936583


Hybrid search

In [12]:
faiss_full = FaissResearch(model_name="sergeyzh/BERTA")
faiss_full.index(texts, ids)                
doc_embs      = faiss_full.embeddings       
id2pos: dict  = {doc_id: i for i, doc_id in enumerate(ids)}


In [13]:
def hybrid_search_fast(query: str, N: int = 100, top_k: int = 10):
    top_tfidf = tfidf_lemma.search(query, top_k=N)
    sub_idx   = np.array([id2pos[d] for d, _ in top_tfidf], dtype=np.int64)

    q_vec = faiss_full._get_embeddings([query])   

    scores = np.squeeze(q_vec @ doc_embs[sub_idx].T)

    order  = np.argsort(scores)[::-1][:top_k]
    result = [(ids[sub_idx[i]], float(scores[i])) for i in order]
    return result


class HybridFastBackend:
    def __init__(self, N=100, top_k=10):
        self.N = N
        self.top_k = top_k
    def index(self, *args, **kwargs):
        pass                         
    def search(self, q, top_k=10):
        return hybrid_search_fast(q, N=self.N, top_k=top_k)

In [15]:
hybrid_fast = HybridFastBackend(N=200)

df_metrics = run_and_log(
    backend      = hybrid_fast,
    q_groups     = q_groups,
    backend_name = "hybrid_fast",
    test_name    = "Hybrid Faiss + BERTA, dim=768 + BM25, N=50",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

                                    test_name      backend   level  \
0  Hybrid Faiss + BERTA, dim=768 + BM25, N=50  hybrid_fast    easy   
1  Hybrid Faiss + BERTA, dim=768 + BM25, N=50  hybrid_fast  medium   
2  Hybrid Faiss + BERTA, dim=768 + BM25, N=50  hybrid_fast    hard   
3  Hybrid Faiss + BERTA, dim=768 + BM25, N=50  hybrid_fast     all   
4  Hybrid Faiss + BERTA, dim=768 + BM25, N=50  hybrid_fast     all   

   Precision@10  Recall@10       MRR  
0      0.098802   0.988024  0.958084  
1      0.098802   0.988024  0.959182  
2      0.096988   0.969880  0.957831  
3      0.098200   0.982000  0.958367  
4      0.098200   0.982000  0.958367  


Named entity recognition improvement

In [18]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsNERTagger
from tqdm.auto import tqdm

segmenter = Segmenter()
emb       = NewsEmbedding()
ner       = NewsNERTagger(emb)

def extract_ents(text: str) -> list[str]:
    doc = Doc(text[:600])
    doc.segment(segmenter)
    doc.tag_ner(ner)
    return [f"{s.type}:{s.text.lower()}" for s in doc.spans]


from processing.faiss_index import FaissResearch 

faiss_full = FaissResearch(model_name="sergeyzh/BERTA")  
faiss_full.index(texts, ids)         


faiss_full.id2ents = {
    str(ids[i]): extract_ents(texts[i])
    for i in tqdm(range(len(ids)), desc="NER payload")
}

NER payload: 100%|██████████| 167/167 [00:01<00:00, 101.05it/s]


In [21]:
def hybrid_search_fast(query: str, N: int = 100, top_k: int = 10):
    top_tfidf = tfidf_lemma.search(query, top_k=N)
    sub_idx   = np.array([id2pos[d] for d, _ in top_tfidf], dtype=np.int64)

    q_vec  = faiss_full._get_embeddings([query])        
    scores = np.squeeze(q_vec @ doc_embs[sub_idx].T)    

    order  = np.argsort(scores)[::-1][:top_k]
    return [(ids[sub_idx[i]], float(scores[i])) for i in order]

class HybridFastBackend:
    def __init__(self, N=100):
        self.N = N
    def index(self, *args, **kwargs):          
        pass
    def search(self, q, top_k=10):
        return hybrid_search_fast(q, self.N, top_k)


def hybrid_search_fast_ner(query: str, N: int = 100, top_k: int = 10,
                           alpha: float = 0.2):
    top_tfidf = tfidf_lemma.search(query, top_k=N)
    sub_idx   = np.array([id2pos[d] for d, _ in top_tfidf], dtype=np.int64)

    q_vec  = faiss_full._get_embeddings([query])
    scores = np.squeeze(q_vec @ doc_embs[sub_idx].T)

    ents_q = set(extract_ents(query))

    for j, doc_idx in enumerate(sub_idx):
        overlap = len(ents_q & set(faiss_full.id2ents.get(ids[doc_idx], [])))
        scores[j] += alpha * overlap

    order  = np.argsort(scores)[::-1][:top_k]
    return [(ids[sub_idx[i]], float(scores[i])) for i in order]

class HybridFastNERBackend(HybridFastBackend):
    def __init__(self, N=100, alpha=0.2):
        super().__init__(N)
        self.alpha = alpha
    def search(self, q, top_k=10):
        return hybrid_search_fast_ner(q, self.N, top_k, self.alpha)

In [26]:
hybrid_ner = HybridFastNERBackend(N=200, alpha=0.3)
faiss_full.id2ents = {
    str(ids[i]): extract_ents(texts[i])
    for i in tqdm(range(len(ids)), desc="NER payload")
}

df_metrics = run_and_log(
    backend      = hybrid_ner,
    q_groups     = q_groups,
    backend_name = "hybrid_ner",
    test_name    = "Hybrid Faiss + BERTA, dim=768 + BM25, N=50 + NER with a=0.3",
    top_k        = 10,
    max_workers  = 20,
)

print(df_metrics)

NER payload: 100%|██████████| 167/167 [00:01<00:00, 99.00it/s] 


                                           test_name     backend   level  \
0  Hybrid Faiss + BERTA, dim=768 + BM25, N=50 + N...  hybrid_ner    easy   
1  Hybrid Faiss + BERTA, dim=768 + BM25, N=50 + N...  hybrid_ner  medium   
2  Hybrid Faiss + BERTA, dim=768 + BM25, N=50 + N...  hybrid_ner    hard   
3  Hybrid Faiss + BERTA, dim=768 + BM25, N=50 + N...  hybrid_ner     all   
4  Hybrid Faiss + BERTA, dim=768 + BM25, N=50 + N...  hybrid_ner     all   

   Precision@10  Recall@10       MRR  
0      0.099401   0.994012  0.963181  
1      0.098204   0.982036  0.966816  
2      0.096386   0.963855  0.933735  
3      0.098000   0.980000  0.954619  
4      0.098000   0.980000  0.954619  


Hybrid search with augmented query (like in prod)

In [ ]:
from __future__ import annotations
import os, re, time, requests
from typing import List, Tuple, Dict

import numpy as np
import faiss
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from huggingface_hub import InferenceClient
from tqdm.auto import tqdm


HF_MODEL       = "sergeyzh/BERTA"
HF_TOKEN       = os.getenv("HF_TOKEN")     
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY") 
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"

CAND_K       = 100   
HF_BATCH     = 32        
HF_RETRIES   = 3       
TOP_K_EVAL   = 10    


def clean_text(txt: str) -> str:
    return re.sub(r"[^а-яёa-z0-9\s]+", " ", txt.lower()).strip()

def gpt_augment(q: str, k: int = 3) -> List[str]:
    if not OPENROUTER_KEY:
        return [q]
    sys_prompt = (
        "Ты ассистент для расширения и исправления запроса пользователя к новостному порталу vesti.ru, чтобы улучшить поиск"
        "Добавь альтернативные варианты к запросу, поправив орфографию если надо, дополнив смысл, добавляя деталей нужных"
        "Если запрос явно с опечаткой, поменяй на наиболее вероятную"
        f"Дай {k} альтернатив через запятую. Без лишних комментариев"
    )
    payload = {
        "model": "openai/gpt-4o-mini",
        "messages": [
            {
                "role": "system",
                "content": sys_prompt,
            },
            {"role": "user", "content": q},
        ],
        "temperature": 0,
        "max_tokens": 64,
        "seed": 42,
    }
    headers = {"Authorization": f"Bearer {OPENROUTER_KEY}",
               "Content-Type": "application/json"}
    try:
        r = requests.post(OPENROUTER_URL, headers=headers,
                          json=payload, timeout=15)
        r.raise_for_status()
        alts = r.json()["choices"][0]["message"]["content"].split(",")
        return [q] + [a.strip() for a in alts[:k]]
    except Exception as e:
        print("GPT-augmentation skipped:", e)
        return [q]


bm25_tokens = [word_tokenize(clean_text(t)) or ["dummy"] for t in tqdm(texts)]
bm25        = BM25Okapi(bm25_tokens)

hf_client = InferenceClient(model=HF_MODEL, token=HF_TOKEN)

def embed_many(
    all_texts: List[str],
    batch: int = HF_BATCH,
    max_retries: int = HF_RETRIES,
) -> np.ndarray:
    vectors: List[List[float]] = []

    for start in tqdm(range(0, len(all_texts), batch),
                      desc="HF batches", leave=False):
        chunk = all_texts[start:start + batch]

        for attempt in range(1, max_retries + 1):
            try:
                feats = hf_client.feature_extraction(chunk)


                if isinstance(feats, np.ndarray):
                    if feats.size == 0:
                        raise ValueError("empty ndarray")
                    feats = feats.tolist()

                if len(feats) != len(chunk):       
                    raise ValueError("incomplete batch")

                vectors.extend(feats)
                break                              

            except Exception as exc:
                if attempt == max_retries:
                    raise RuntimeError(
                        f"HF embedding failed after {max_retries} retries "
                        f"for batch starting {start}: {exc}"
                    ) from exc
                time.sleep(1.5 * attempt)        

    arr = np.asarray(vectors, dtype=np.float32)
    arr /= np.linalg.norm(arr, axis=1, keepdims=True) + 1e-12
    return arr

emb_arr = embed_many(texts)
index_faiss = faiss.IndexFlatL2(emb_arr.shape[1])
index_faiss.add(emb_arr)

id2pos = {doc_id: idx for idx, doc_id in enumerate(ids)}


class HybridAugBackend:
    """BM-25 candidate + cosine FAISS rerank + optional GPT expansion"""

    def _embed_query(self, q: str) -> np.ndarray:
        vec = hf_client.feature_extraction([q])[0]
        vec = np.asarray(vec, dtype=np.float32)
        vec /= np.linalg.norm(vec) + 1e-12
        return vec

    def search(self, q: str, top_k: int = 10) -> List[Tuple[str, float]]:
        variants = gpt_augment(q, k=3)
        pool: Dict[str, float] = {}

        for v in variants:
            scores = bm25.get_scores(word_tokenize(clean_text(v)))
            for i in np.argsort(scores)[::-1][:CAND_K]:
                did = ids[i]
                pool[did] = max(pool.get(did, 0.0), scores[i])

        if not pool:
            return []

        cand_ids = list(pool)
        emb_subset = emb_arr[[id2pos[d] for d in cand_ids]]

        q_vec = self._embed_query(q)
        cos   = emb_subset @ q_vec
        best  = cos.argsort()[::-1][:top_k]
        return [(cand_ids[i], float(cos[i])) for i in best]


backend = HybridAugBackend()

df_metrics = run_and_log(
    backend      = backend,
    q_groups     = q_groups,
    backend_name = "hybrid_gpt_aug",
    test_name    = "Hybrid BM25→FAISS + GPT-aug",
    top_k        = TOP_K_EVAL,
    max_workers  = 20,
)

df_metrics


100%|██████████| 167/167 [00:00<00:00, 1476.75it/s]


,test_name,backend,level,Precision@10,Recall@10,MRR
0,Hybrid BM25→FAISS + GPT-aug,hybrid_gpt_aug,easy,0.099401,0.994012,0.994012
1,Hybrid BM25→FAISS + GPT-aug,hybrid_gpt_aug,medium,0.098204,0.982036,0.982036
2,Hybrid BM25→FAISS + GPT-aug,hybrid_gpt_aug,hard,0.098193,0.981928,0.981928
3,Hybrid BM25→FAISS + GPT-aug,hybrid_gpt_aug,all,0.098600,0.986000,0.986000
4,Hybrid BM25→FAISS + GPT-aug,hybrid_gpt_aug,all,0.098600,0.986000,0.986000
